# Test plane-based registration
Test different plane-based registration methods

In [32]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import get_correspondences, extract_corresponding_features
from planeslam.geometry.util import quat_to_rot_mat

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load AirSim data

In [33]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [34]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

### Decoupled

In [36]:
from planeslam.registration import decoupled_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[ 4.17561936e+01]
 [-6.91389945e-02]
 [ 2.35085002e+02]]
[[-4.93224409e+01]
 [ 6.85646618e-14]
 [ 5.25042913e+00]]
[[-73.93652591]
 [ -0.26256389]
 [  8.19982805]]
[[ 3.96851642]
 [ 0.02358479]
 [-0.21498566]]
[[ 3.98560873e+00]
 [-1.96732949e-03]
 [-1.28063227e-01]]
[[ 2.30098658]
 [-0.00383921]
 [-0.02968871]]
[[ 4.49636042]
 [-0.01993057]
 [-0.03203159]]
[[ 4.86525903]
 [ 0.00808046]
 [-0.05318259]]
[[ 4.38436669e+00]
 [-3.92347594e-03]
 [-7.51966504e-03]]
[[ 4.28062767e+00]
 [-2.01415903e-01]
 [ 8.76014594e-04]]
[[ 4.51646742]
 [ 0.14195673]
 [-0.08669257]]
[[ 1.12253149e+01]
 [-8.96304618e-03]
 [-8.63307625e+00]]
[[-12.44248494]
 [ -0.17425084]
 [ 45.92897225]]
[[1.22454902e+03]
 [3.76937261e+00]
 [5.35459938e+03]]
[[-0.56798381]
 [ 0.06901303]
 [-0.10303344]]
[[-3.28092888e+01]
 [-1.45730019e-02]
 [ 1.48726002e+01]]
[[0.54583353]
 [0.02485906]
 [0.03897516]]
[[ 0.05576875]
 [-0.02552299]
 [-0.0209569 ]]
[[-0.25355929]
 [ 0.005749  ]
 [ 0.03889906]]
average registration time:  0.

### Gauss-newton

In [37]:
from planeslam.registration import GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = GN_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[30.65781317]
 [11.14045758]
 [24.75049124]]
[[-2.30432372]
 [ 0.41884598]
 [ 2.66917582]]
[[32.10861411]
 [-1.59487608]
 [-2.7139839 ]]
[[ 1.13175403]
 [-1.19555816]
 [-0.70387388]]
[[ 4.46552276]
 [-2.58861795]
 [ 0.43554267]]
[[ 2.57591884]
 [-1.27890639]
 [ 1.22939859]]
[[ 1.01337052]
 [ 4.24488642]
 [-1.43561391]]
[[ 1.88583322]
 [ 4.58631592]
 [-4.81763078]]
[[ 1.66215405]
 [ 0.64699314]
 [-0.33533968]]
[[0.38770594]
 [2.62742646]
 [0.32078439]]
[[0.36142706]
 [2.24109746]
 [1.70968005]]
[[  7.17623909]
 [ 29.53666312]
 [-52.35872499]]
[[8.69197295]
 [0.06868411]
 [2.63284572]]
[[8.61138632]
 [0.45126762]
 [6.60660178]]
[[0.59507523]
 [1.97110034]
 [0.14642334]]
[[0.45874694]
 [1.2864742 ]
 [2.1262965 ]]
[[-0.97621036]
 [ 0.07816455]
 [ 0.05089134]]
[[0.16688146]
 [0.0009609 ]
 [0.02947214]]
[[-0.03333937]
 [ 0.21852243]
 [ 1.64766113]]
average registration time:  0.0017640214217336553


### Torch GN

In [38]:
# Set torch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [45]:
from planeslam.registration import torch_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = torch_GN_register(scans[i+1], scans[i], device)
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[ 6.8910704]
 [-1.8441594]
 [-2.9303555]]
[[ 2.2052667 ]
 [-0.12693232]
 [ 2.5371797 ]]
[[ 7.477677  ]
 [-0.1557581 ]
 [-0.47617593]]
[[ 3.966186  ]
 [ 0.02562693]
 [-0.18225062]]
[[ 3.9931989 ]
 [ 0.02337583]
 [-0.13810912]]
[[ 1.4888978 ]
 [-0.0102578 ]
 [-0.01005685]]
[[ 4.4937363 ]
 [-0.01709931]
 [-0.04390385]]
[[ 3.1681662 ]
 [-0.35375258]
 [-5.063493  ]]
[[ 4.4134665 ]
 [-0.02657721]
 [-0.05436486]]
[[ 4.301298  ]
 [-0.20272158]
 [ 0.0473389 ]]
[[ 4.5172296 ]
 [ 0.13417937]
 [-0.10137098]]
[[ 3.3971615 ]
 [-0.02664175]
 [-0.30003643]]
[[ 1.5254318 ]
 [-0.03821417]
 [ 1.090883  ]]
[[ 8.536132 ]
 [-0.0620997]
 [-0.7797686]]
[[-0.5654953 ]
 [ 0.07376771]
 [-0.10531069]]
[[-0.5066407 ]
 [ 0.07357979]
 [ 2.7039537 ]]
[[0.5499096 ]
 [0.02354406]
 [0.03654559]]
[[ 0.05589207]
 [-0.02576118]
 [-0.01947335]]
[[-0.25309563]
 [ 0.00926378]
 [ 0.04578644]]
average registration time:  0.10787519655729595


In [46]:
np.round(abs_traj_transformations[:,:3,3],2)

array([[  6.89,  -1.84,  -2.93],
       [ -3.41,  -1.97,  -2.41],
       [  3.97,  -2.14,  -2.74],
       [  7.9 ,  -2.12,  -2.96],
       [ 11.88,  -2.04,  -3.16],
       [ 13.39,  -2.  ,  -3.13],
       [ 17.86,  -2.08,  -3.21],
       [ 10.33,  -3.1 , -21.64],
       [ 14.59,  -3.13, -21.77],
       [ 19.04,  -3.39, -21.62],
       [ 22.18,  -3.27, -22.85],
       [ 19.33,  -3.35, -27.87],
       [-32.3 ,  -3.66,   3.11],
       [-23.45,  -3.09,  -6.64],
       [-24.47,  -2.88,  -4.92],
       [ -0.97,  -2.72,  27.67],
       [  0.86,  -2.68,  27.73],
       [ -0.5 ,  -2.66,  27.72],
       [ -1.45,  -2.7 ,  27.74]])